# Truncated factorization causal bootstrapping (Algorithm 3)

This notebook implements **Algorithm 3** from *Causal bootstrapping* (Little & Badawy, 2020).  
It applies when **all variables are observed** and the causal graph is known, so that truncated factorization can be used.

Algorithm 3 weights (paper):
- Let \(P(X)\) be parents of effect variable(s) \(X\)
- Let \(E = P(X) \setminus \{Y\}\)
\[
\bar w_i = \frac{\prod_{v\in E} \hat p(v_i\mid P(v) \text{ with } Y=y^*)}{\hat p(P(X)=p_i \text{ with } Y=y^*)}
\]
\[
w_i = \frac{1}{N} K[y_i-y^*] \bar w_i \quad (\text{if } Y\in P(X))
\]

Key reference: Algorithm 3 and Eq. (7)-(8), plus Section 2.5 in the paper. fileciteturn0file0


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

def _ensure_2d(arr):
    arr = np.asarray(arr)
    if arr.ndim == 1:
        arr = arr.reshape(-1, 1)
    return arr

def gaussian_kernel_matrix(A, B=None, bandwidth=1.0):
    """
    Gaussian/RBF kernel matrix: K_ij = exp(-0.5 * ||a_i - b_j||^2 / h^2)
    A: (n,d), B: (m,d)
    """
    A = _ensure_2d(A)
    if B is None:
        B = A
    else:
        B = _ensure_2d(B)
    dists = cdist(A, B, metric="euclidean")
    return np.exp(-0.5 * (dists / float(bandwidth)) ** 2)

def kernel_y_vector(y_data, y_star, *, discrete=True, bandwidth_y=1.0):
    """
    K[y_i - y*] used in the paper.
    - discrete=True -> Kronecker delta (1 if equal else 0)
    - discrete=False -> Gaussian kernel on (y_i - y*)
    """
    y_data = np.asarray(y_data)
    if discrete:
        return (y_data == y_star).astype(float)
    return np.exp(-0.5 * ((y_data - y_star) / float(bandwidth_y)) ** 2)

def phat_y_given_S(y_data, S_data, y_star, *, discrete_y=True, bandwidth_S=1.0, bandwidth_y=1.0, eps=1e-12):
    """
    Nonparametric estimate of p_hat(y* | S_i) for each i, using kernel regression:
      p_hat(y*|S_i) = sum_j K_S(S_i, S_j) * K_Y(y_j, y*) / sum_j K_S(S_i, S_j)
    Returns: (N,) vector over i.
    """
    K_S = gaussian_kernel_matrix(S_data, bandwidth=bandwidth_S)  # (N,N)
    K_Y = kernel_y_vector(y_data, y_star, discrete=discrete_y, bandwidth_y=bandwidth_y)  # (N,)
    numer = K_S @ K_Y
    denom = K_S.sum(axis=1)
    return numer / np.maximum(denom, eps)

def phat_z_given_y(z_data, y_data, *, bandwidth_z=1.0, eps=1e-12):
    """
    For front-door (Algorithm 2): estimate p_hat(z_i | y=v) for all i and each discrete y=v.
    KDE on z within each y-group:
      p_hat(z_i | y=v) ∝ (1/N_v) * sum_{j:y_j=v} K_z(z_i, z_j)
    Returns: dict mapping y_value -> (N,) vector, entry i is p_hat(z_i | y_value)
    """
    z = _ensure_2d(z_data)
    y = np.asarray(y_data)
    y_vals = np.unique(y)

    Kzz = gaussian_kernel_matrix(z, bandwidth=bandwidth_z)  # (N,N)
    phat = {}
    for v in y_vals:
        mask = (y == v)
        Nv = int(mask.sum())
        if Nv == 0:
            phat[v] = np.full(len(y), eps)
            continue
        numer = Kzz[:, mask].sum(axis=1) / float(Nv)
        phat[v] = np.maximum(numer, eps)
    return phat


## Provide the causal graph inputs

You must specify:

- `y_col`: intervention variable **Y** (usually your target label)
- `x_cols`: features **X** you want to sample (used only for output; the weighting uses `x_parents`)
- `x_parents`: the parent set **P(X)** used in the truncated factorization formula (a list of column names).
  - **Important:** include `y_col` in `x_parents` if Y is a parent of X in your graph.
- `parents`: a dictionary mapping each variable name `v` in `E` to its parent list `P(v)` (may include `y_col`).

Assumptions:
- Variables are treated as continuous for kernels unless included in `discrete_vars`.
- This is a *nonparametric* estimator intended to mirror the paper's RKHS/KDE simplification.


In [ ]:
# --- USER INPUTS ---
df = pd.read_csv("heart_disease_preprocessed.csv")  # change if needed

y_col = "heartdiseasepresence"

# Example (EDIT to match your causal DAG):
# Suppose the feature block X depends on Y and some observed covariates E.
x_cols = [c for c in df.columns if c != y_col]

# Parents of the (vector) feature block X in your DAG:
x_parents = [y_col, "age", "sex_Female", "sex_Male"]  # EDIT: P(X) in your graph

# E = P(X) \ {Y}:
E_vars = [v for v in x_parents if v != y_col]

# Parent sets for each v in E (EDIT to match your graph).
# If a v has no parents, use [].
parents = {
    "age": [],
    "sex_Female": [],
    "sex_Male": [],
}

# Kernel settings
discrete_vars = {y_col, "sex_Female", "sex_Male"}  # treat these with Kronecker delta
bandwidth = 1.0  # shared bandwidth for continuous kernels

random_seed = 0

print("y_col:", y_col)
print("x_parents:", x_parents)
print("E_vars:", E_vars)
print("parents(v):", parents)
print("N:", len(df))

In [ ]:
def _phat_joint(query, data, *, bandwidth=1.0, eps=1e-12):
    """Unnormalized KDE joint density at query points: p_hat(query) ∝ (1/N) sum_j K(query, data_j)."""
    K = gaussian_kernel_matrix(query, B=data, bandwidth=bandwidth)  # (Q,N)
    return np.maximum(K.mean(axis=1), eps)

def _kernel_target_matrix(target_data, query_targets, *, discrete=False, bandwidth=1.0):
    """Matrix Ky[q,j] = K(query_targets[q] - target_data[j])."""
    target_data = np.asarray(target_data)
    qt = np.asarray(query_targets)
    if qt.ndim == 0:
        qt = qt.reshape(1)
    if discrete:
        return (qt[:, None] == target_data[None, :]).astype(float)
    return np.exp(-0.5 * ((qt[:, None] - target_data[None, :]) / float(bandwidth)) ** 2)

def _phat_conditional(target_data, parent_data, *, query_targets, query_parents,
                      target_discrete=False, bandwidth_parent=1.0, bandwidth_target=1.0, eps=1e-12):
    """Kernel regression estimate of p_hat(target=query_targets | parents=query_parents).

    Vectorized over Q queries (typically Q=N).
    """
    parent_data = _ensure_2d(parent_data)
    query_parents = _ensure_2d(query_parents)

    # K(parents): (Q,N)
    Kp = gaussian_kernel_matrix(query_parents, B=parent_data, bandwidth=bandwidth_parent)

    # K(target): (Q,N)
    Ky = _kernel_target_matrix(target_data, query_targets, discrete=target_discrete, bandwidth=bandwidth_target)

    numer = (Kp * Ky).sum(axis=1)
    denom = Kp.sum(axis=1)
    return numer / np.maximum(denom, eps)

def causal_bootstrap_truncated_factorization(df, *, x_cols, y_col, x_parents, parents,
                                             discrete_vars=None, bandwidth=1.0,
                                             random_seed=0, eps=1e-12):
    """Truncated factorization causal bootstrap (Algorithm 3).

    Returns dataframe with columns x_cols + [y_col], same number of rows as df.
    """
    discrete_vars = set(discrete_vars or [])
    N = len(df)
    rng = np.random.default_rng(random_seed)

    y = df[y_col].to_numpy()
    y_vals = np.unique(y)

    PX_data = df[x_parents].to_numpy(dtype=float)  # observational P(X)

    out_rows = []
    for y_star in y_vals:
        n_star = int((y == y_star).sum())
        if n_star == 0:
            continue

        # Query P(X) at each row's E values but with Y fixed to y_star
        PX_query_df = df[x_parents].copy()
        if y_col in x_parents:
            PX_query_df[y_col] = y_star
        PX_query = PX_query_df.to_numpy(dtype=float)

        phat_PX = _phat_joint(PX_query, PX_data, bandwidth=bandwidth, eps=eps)  # (N,)

        # Product over v in E = P(X) \ {Y}
        prod = np.ones(N, dtype=float)
        for v in [vv for vv in x_parents if vv != y_col]:
            Pv = parents.get(v, [])
            v_data = df[v].to_numpy()
            if len(Pv) == 0:
                # marginal p(v_i)
                v_mat = _ensure_2d(v_data.astype(float))
                phat_v = _phat_joint(v_mat, v_mat, bandwidth=bandwidth, eps=eps)
            else:
                parent_query_df = df[Pv].copy()
                if y_col in Pv:
                    parent_query_df[y_col] = y_star
                parent_query = parent_query_df.to_numpy(dtype=float)

                parent_data = df[Pv].to_numpy(dtype=float)  # observational parent data
                phat_v = _phat_conditional(
                    target_data=v_data,
                    parent_data=parent_data,
                    query_targets=v_data,           # v_i for each i
                    query_parents=parent_query,     # parents_i with Y=y_star if applicable
                    target_discrete=(v in discrete_vars),
                    bandwidth_parent=bandwidth,
                    bandwidth_target=bandwidth,
                    eps=eps
                )
            prod *= np.maximum(phat_v, eps)

        w_bar = prod / np.maximum(phat_PX, eps)

        # If Y is in P(X), include K[y_i - y*]
        Ky = kernel_y_vector(y, y_star, discrete=(y_col in discrete_vars))
        w = (Ky * w_bar) / float(N)

        w_sum = w.sum()
        if w_sum <= 0:
            continue
        p = w / w_sum
        idx = rng.choice(np.arange(N), size=n_star, replace=True, p=p)

        block = df.iloc[idx][x_cols].copy()
        block[y_col] = y_star
        out_rows.append(block)

    df_star = pd.concat(out_rows, ignore_index=True)
    if len(df_star) != N:
        df_star = df_star.sample(n=N, replace=True, random_state=random_seed).reset_index(drop=True)
    return df_star


In [ ]:
df_tf = causal_bootstrap_truncated_factorization(
    df,
    x_cols=x_cols,
    y_col=y_col,
    x_parents=x_parents,
    parents=parents,
    discrete_vars=discrete_vars,
    bandwidth=bandwidth,
    random_seed=random_seed,
)

df_tf.head(), df_tf.shape

In [ ]:
out_path = "heart_disease_preprocessed_truncated_factorization_BOOTSTRAPPED.csv"
df_tf.to_csv(out_path, index=False)
out_path